# 데이터 로드

In [24]:
import numpy as np
import pandas as pd

Combined_Review_origin = pd.read_excel('combined_review.xlsx')
Combined_Review_origin.shape # (2157,11)
Combined_Review_origin.head()

,회사명,작성일,별점,제목_본문
0,toto,2022-01-25,5,"Amazing product.I travel a lot, and I frequent..."
1,toto,2022-08-28,5,Husband says best gift ever!.I purchased this ...
2,toto,2018-07-25,3,"Leakage problem fix.7/24/18First of, I did not..."
3,biobidet,2021-12-21,3,Loved it for the few days it has worked..We bo...
4,biobidet,2021-03-01,5,It Works Beautifully.It does everything it say...


In [25]:
# 카피본 떠놓고 시작하기
combined_review = Combined_Review_origin.copy()
combined_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31198 entries, 0 to 31197
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   회사명     31198 non-null  object        
 1   작성일     31198 non-null  datetime64[ns]
 2   별점      31198 non-null  int64         
 3   제목_본문   31198 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 975.1+ KB


# 데이터 전처리

In [26]:
combined_review['제목_본문']

0        Amazing product.I travel a lot, and I frequent...
1        Husband says best gift ever!.I purchased this ...
2        Leakage problem fix.7/24/18First of, I did not...
3        Loved it for the few days it has worked..We bo...
4        It Works Beautifully.It does everything it say...
                               ...                        
31193    Good product.I installed BB-2000 to replace my...
31194    Fantastic product.We have had our current For ...
31195    Great product.I bought this product about two ...
31196    Straight forward installation.This is  second ...
31197    Wouldn’t be without it.We just recently purcha...
Name: 제목_본문, Length: 31198, dtype: object

In [27]:
# !pip install transformers

In [28]:
# !pip install torch

In [29]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import re
import time
import datetime
import nltk
from nltk.corpus import stopwords

In [30]:
# 파이썬 경고 메세지 출력 안하기
import warnings
warnings.filterwarnings(action='ignore')

In [31]:
nltk.download('stopwords') # 구두점 사전 다운로드 / 구두점: 글의 여러 가지 경계를 구분하기 위해 사용되는 반점(,), 온점(.), 물음표(?), 쌍점( : ), 쌍반점(;), 줄표(--), 붙임표(-) 등의 부호
nltk.download('punkt') # 불용어 사전 다운로드 / 불용어: 문장을 분석하는데 큰 의미가 없는 문자열

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
# # Download stopwords
# nltk.download('stopwords')
# stop_words = stopwords.words("english")

# # Define preprocess function
# def preprocess(text):
#     # Convert to lowercase
#     text = text.lower()
#     # Remove non-alphanumeric characters and punctuation
#     text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#     # Remove words with length less than 2
#     text = " ".join([word for word in text.split() if len(word) > 1])
#     return text

# # Define a function that maps review scores to three labels
# def score_to_label(score):
#     if score >= 4:
#         return "positive"
#     elif score == 3:
#         return "neutral"
#     else:
#         return "negative"

# # Load dataset
# combined_review = pd.read_excel('combined_review.xlsx')

# # Preprocess reviews
# combined_review['제목_본문'] = combined_review['제목_본문'].apply(preprocess)
# combined_review['별점'] = combined_review['별점'].apply(score_to_label)

# # Load the pre-trained BERT model and tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)

# # Tokenize the reviews and create input sequences
# reviews = combined_review['제목_본문'].values
# labels = combined_review['별점'].values

# input_ids = []
# attention_masks = []

# for review in reviews:
#     encoded_dict = tokenizer.encode_plus(
#                         review,
#                         add_special_tokens=True,
#                         max_length=128,
#                         pad_to_max_length=True,
#                         return_attention_mask=True,
#                         return_tensors='pt',
#                         stopwords=stop_words,
#                         truncation=True
#                    )
#     input_ids.append(encoded_dict['input_ids'])
#     attention_masks.append(encoded_dict['attention_mask'])

# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(labels)

# # Split the dataset into training and validation sets
# dataset = TensorDataset(input_ids, attention_masks, labels)
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# # Create data loaders for training and validation sets
# batch_size = 32
# train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
# val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

In [44]:
# Download stopwords
nltk.download('stopwords')
stop_words = stopwords.words("english")

# Define preprocess function
def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    # Remove non-alphanumeric characters and punctuation
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    # Remove words with length less than 2
    text = " ".join([word for word in text.split() if len(word) > 1])
    return text

# Define a function that maps review scores to three labels
def score_to_label(score):
    if score >= 4:
        return 2
    elif score == 3:
        return 1
    else:
        return 0

# Load dataset
combined_review = pd.read_excel('combined_review.xlsx')

# Preprocess reviews
combined_review['제목_본문'] = combined_review['제목_본문'].apply(preprocess)
combined_review['별점'] = combined_review['별점'].apply(score_to_label)

sentences = combined_review['제목_본문']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# 리뷰 추출
sentences = combined_review['제목_본문']
sentences = ["[CLS]" + str(review) + "[SEP]" for review in combined_review['제목_본문']]
sentences[:10]

['[CLS]amazing producti travel lot and frequently stay at hotels in asia that feature bidet style toilet seat have long considered installing one at home and finally overcame inertia and purchased the washlet c2 this is an amazing product it makes this very necessary personal chore more pleasant more hygienic and dare say it more fun installation is very easy recommend this version with the remote control for ease of use units with the controls mounted on the side of the seat require phenomenal degree of agility to operate this washlet model was neither the cheapest nor the most expensive but think it is the highest value for money am totally impressed by the high degree of design and build quality in this product which says lot because am tough to impress engineer[SEP]',
 '[CLS]husband says best gift everi purchased this for my husband who fell in love with this product at hotel it was easy to install and he loves this seat weve had it for few months and he continues to report it is t

In [45]:
# 별점 추출
labels = combined_review['별점'].values
labels

array([2, 2, 1, ..., 2, 2, 2], dtype=int64)

In [37]:
# BERT의 토크나이저로 문장을 토큰으로 분리
# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sentence) for sentence in sentences]

print(sentences[0])
print(tokenized_texts[0])

[CLS]amazing producti travel lot and frequently stay at hotels in asia that feature bidet style toilet seat have long considered installing one at home and finally overcame inertia and purchased the washlet c2 this is an amazing product it makes this very necessary personal chore more pleasant more hygienic and dare say it more fun installation is very easy recommend this version with the remote control for ease of use units with the controls mounted on the side of the seat require phenomenal degree of agility to operate this washlet model was neither the cheapest nor the most expensive but think it is the highest value for money am totally impressed by the high degree of design and build quality in this product which says lot because am tough to impress engineer[SEP]
['[CLS]', 'ama', '##zing', 'product', '##i', 'travel', 'lot', 'and', 'frequently', 'stay', 'at', 'hotels', 'in', 'asi', '##a', 'that', 'feature', 'bid', '##et', 'style', 'toile', '##t', 'seat', 'have', 'long', 'considered

In [38]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  28149,  19308,  21535,  10116,  23595,  19826,  10111,
        28846,  29597,  10160,  82102,  10106,  21744,  10113,  10189,
        19072,  50385,  10308,  13351,  63131,  10123,  17687,  10529,
        11695,  14289,  50562,  11945,  10464,  10160,  11816,  10111,
        21256,  10491,  67354,  10112,  10106,  49366,  10113,  10111,
        24463,  10105,  10134,  41058,  10123,    171,  10729,  10531,
        10124,  10151,  28149,  19308,  21535,  10271,  20562,  10531,
        12558,  27039,  14927,  11257,  10246,  10798,  20648,  42658,
        11236,  10798,  15165,  55713,  11130,  10111,  45476,  23763,
        10271,  10798,  41807,  38639,  10124,  12558,  44346,  11639,
        22530, 101480,  10531,  11674,  10169,  10105,  55808,  12608,
        10142,  22257,  10341,  10108,  11760,  17163,  10169,  10105,
        59897,  44709,  10135,  10105,  12250,  10108,  10105,  17687,
        35742,  99142, 102523,  10161,  16455,  10108,  16942,  44704,
      

In [39]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [46]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=324, test_size=0.2)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=324, 
                                                       test_size=0.2)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101, 10192, 20390, 44346, 50562, 91177, 12563, 10157, 20648, 42658,
        10336, 24463, 10551, 10142, 17446, 11816, 10111, 11059, 10464, 10142,
        15127, 10312, 11155, 10301, 12403, 13190, 10111, 15127, 43310, 10106,
        10105, 27657, 10319, 15433, 10551, 33003, 37247, 10107, 14009, 14772,
        10142, 10911, 10111, 10105, 13190, 22153, 10271, 14009, 15198, 10142,
        11345,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [47]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## GPU

In [51]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

SystemError: GPU device not found

In [49]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [52]:
# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [53]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
x = 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_data, train_size=, random_state=10)

In [ ]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

In [ ]:

# 훈련셋의 앞부분 출력
train.head(10)

In [ ]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

In [ ]:

# 라벨 추출
labels = train['label'].values
labels
     

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])
     

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]